In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix, classification_report
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [2]:
# report_filename = 'model_100_epochs_evaluation.csv'               # initial one
# report_filename = 'model_500_samples_paper_evaluation.csv'        # first with 500 samples
report_filename = 'model_500_samples_paper_v5_evaluation.csv'     # the chosen one

report_filename_orig = 'model_500_samples_orig_configs_evaluation.csv'   # again with original configs from first one


report_path = os.path.join('..', 'analysis', 'finetuned', report_filename)
df = pd.read_csv(report_path).set_index('filepath')


report_path_orig = os.path.join('..', 'analysis', 'finetuned', report_filename_orig)
df_orig = pd.read_csv(report_path_orig).set_index('filepath')

In [3]:
available_classes = list(df['label'].unique())

available_classes = available_classes[::-1]


In [4]:
available_classes = get_avaliable_classes_alphabetic()

In [5]:
df.columns

Index(['label', 'concrete_plates', 'paving_stones', 'sett', 'asphalt', 'grass',
       'compacted', 'concrete', 'cobblestone', 'gravel', 'ground'],
      dtype='object')

In [6]:
predicted_labels = list(df[available_classes].idxmax(axis=1))


predicted_labels_orig = list(df_orig[available_classes].idxmax(axis=1))

In [7]:
true_labels = list(df['label'])

true_labels_orig = list(df_orig['label'])

In [8]:
assert len(predicted_labels) == len(true_labels)

In [9]:
c_mat = confusion_matrix(true_labels,
                         predicted_labels,
                         labels=available_classes,
                         normalize='true') * 100

c_mat_orig = confusion_matrix(true_labels_orig,
                         predicted_labels_orig,
                         labels=available_classes,
                         normalize='true') * 100

In [10]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes,
                y=available_classes,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [11]:
np.sum(c_mat, axis=0)

array([109. , 100. ,  93.5, 100.5,  98.5, 100.5, 100. ,  99. , 101. ,
        98. ])

In [12]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes,
    output_dict=True
)

{'asphalt': {'precision': 0.8899082568807339,
  'recall': 0.97,
  'f1-score': 0.9282296650717703,
  'support': 200.0},
 'cobblestone': {'precision': 0.95,
  'recall': 0.95,
  'f1-score': 0.95,
  'support': 200.0},
 'compacted': {'precision': 0.9251336898395722,
  'recall': 0.865,
  'f1-score': 0.8940568475452196,
  'support': 200.0},
 'concrete': {'precision': 0.8756218905472637,
  'recall': 0.88,
  'f1-score': 0.8778054862842892,
  'support': 200.0},
 'concrete_plates': {'precision': 0.949238578680203,
  'recall': 0.935,
  'f1-score': 0.9420654911838791,
  'support': 200.0},
 'grass': {'precision': 0.9950248756218906,
  'recall': 1.0,
  'f1-score': 0.9975062344139651,
  'support': 200.0},
 'gravel': {'precision': 0.97,
  'recall': 0.97,
  'f1-score': 0.97,
  'support': 200.0},
 'ground': {'precision': 0.9848484848484849,
  'recall': 0.975,
  'f1-score': 0.9798994974874372,
  'support': 200.0},
 'paving_stones': {'precision': 0.9257425742574258,
  'recall': 0.935,
  'f1-score': 0.93034

In [13]:
c_mat.diagonal().mean()

93.5

In [14]:
c_mat.diagonal().std()

4.544227107000705

In [15]:
fig = px.imshow(c_mat_orig,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes,
                y=available_classes,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [16]:
c_mat_orig.diagonal().mean()

59.55

In [17]:
c_mat_orig.diagonal().std()

26.312972086026317

In [19]:
other_models_records = read_json('../analysis/figures/barplot_data.json')

In [22]:
other_models_records.append(
    {'model_name': 'first_FT','score':c_mat_orig.diagonal().mean(),'std':c_mat_orig.diagonal().std()}
    )

other_models_records.append(
    {'model_name': 'best_FT','score':c_mat.diagonal().mean(),'std':c_mat.diagonal().std()}
)

In [26]:
all_models_df = pd.DataFrame(other_models_records).sort_values('score', ascending=False)

In [27]:
all_models_df

,model_name,score,std
11,best_FT,93.50,4.544227
0,combined_3,60.94,37.172899
1,combined,59.56,38.277179
10,first_FT,59.55,26.312972
2,ViT-bigG-14-CLIPA-336,56.04,37.246723
3,ViT-H-14-378-quickgelu,53.86,29.253519
4,EVA02-E-14-plus,52.20,35.168622
5,ViT-SO400M-14-SigLIP-384,48.64,36.065585
6,ViT-L_14,41.78,37.782266
7,RN101,37.46,28.856757


In [66]:
fig = go.Figure(data=[
    go.Bar(
        x=all_models_df['model_name'],
        y=all_models_df['score'],
        text = all_models_df['score'],
        textposition='inside',
        textangle=45,

        error_y=dict(
            type='data', # or 'percent' for percentage errors
            array=all_models_df['std'],
            visible=True,
        thickness=0.5,
        width=7,     
        )
    )
])

fig.show()